In [1]:
import pandas as pd
import numpy as np
import os #required to list file names in folder to load them into dataframes
import glob #check if this is still required

#Required for Public holidays
from datetime import date
import holidays #dont worry about installing the add-on

In [2]:
df_source_SKUs = pd.read_csv('input_skus_str.csv')
df_SKU_lookup = pd.read_csv('df_SKU_lookup.csv')
df_inv_mast = pd.read_csv('df_inv_mast.csv',encoding='UTF-8')

In [3]:
df_source_SKUs.head()

,0
0,32166
1,32322
2,33146
3,33151
4,33582


In [4]:
df_source_SKUs.shape

(1245, 1)

In [5]:
df_source_SKUs.columns = ["SKUstr"]

In [6]:
df_source_SKUs.head()

,SKUstr
0,32166
1,32322
2,33146
3,33151
4,33582


In [7]:
df_source_SKUs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1245 entries, 0 to 1244
Data columns (total 1 columns):
SKUstr    1245 non-null object
dtypes: object(1)
memory usage: 9.9+ KB


In [8]:
df_SKU_lookup.head()

,SKUstr,SKUint
0,312996,312996
1,323294,323294
2,337140,337140
3,340663,340663
4,356699,356699


In [9]:
df_SKU_lookup.shape

(1270, 2)

In [10]:
df_SKU_lookup.tail(100)

,SKUstr,SKUint
1170,38918,38918
1171,390169,390169
1172,390546,390546
1173,390573,390573
1174,390575,390575
...,...,...
1265,399822,399822
1266,399853,399853
1267,399980,399980
1268,S080501_1500_30_EUR,3169823


In [11]:
df_inv_mast.head()

,SKUint,TotalUnits,TotalRev,AWP,Category
0,32166,1,1569.42,1569.42,LENTES
1,32322,1,134.89,134.89,OTHER
2,33146,1,408.00,408.00,LENTES
3,33151,1,295.00,295.00,OBJETIVOS CÁMARAS DIGITALES
4,33582,1,23.14,23.14,OTHER


In [12]:
# ALREADY WORKING this is the lookup public holidays - which works when you use it with an apply function 
def public_holiday(row): 
    """
    Input: Date
    Output: 1 or 0 if the specific date is a public holiday or not.
    
    The purpose of this function is to determine if the date is a public holiday in Spain (ES)
    """
    es_holidays = holidays.CountryHoliday('ES', years=[2017])
    return(row.date_dt in es_holidays)
#df_working['date_isPublicHol'] = df_working.apply(public_holiday, axis=1).astype(int) #was isPublicHol

In [13]:
#Spits out a single row dataframe with the features extracted.
def dt_feature_extraction(dt):
    
    global df
    
    lst = []
    lst.append([dt])

    df = pd.DataFrame(lst)    
    df.columns = ['Date']    
    
   
    df['date_dt']= pd.to_datetime(df['Date']) #was dttime
    df['date_DoW'] = df['date_dt'].dt.day_name() #was day_of_week
    df['date_DoWnum'] = df['date_dt'].dt.dayofweek
    df['date_DoM'] = df['date_dt'].dt.day # was day_of_month
    df['date_WoY'] = df['date_dt'].dt.week #was WeekNum_of_year
    df['date_isWeekend'] = np.where(df['date_DoW'].isin(['Sunday','Saturday']),1,0) # was isWeekend    
    df['date_isPublicHol'] = df.apply(public_holiday, axis=1).astype(int) #was isPublicHol    

    dummy = pd.get_dummies(df['date_DoW'])

    df = df.merge(dummy, left_index=True , right_index=True)
    
    dict_days = {}
    for i in df['date_DoW'].unique():
        dict_days[i] = 'is' + i  
    
    df.rename(columns=dict_days, inplace=True)
    
    #df.drop(['Date'], axis=1, inplace=True) # This string field is no longer as we have 
                                                #the Day of the Month and we know all transactions occured in March.
    #df.drop(['date_DoW'], axis=1, inplace=True) # No longer needed thanks to Dummy Variables.
    #df.drop(['date_dt'], axis=1, inplace=True)
    
    return df

dt_feature_extraction('2017-04-01')

,Date,date_dt,date_DoW,date_DoWnum,date_DoM,date_WoY,date_isWeekend,date_isPublicHol,isSaturday
0,2017-04-01,2017-04-01,Saturday,5,1,13,1,0,1


In [14]:
df_intermediate = (pd.merge(df_source_SKUs, df_SKU_lookup, on='SKUstr'))

In [15]:
df_intermediate.shape

(1270, 2)

In [16]:
df_final = (pd.merge(df_intermediate, df_inv_mast, on="SKUint"))

In [17]:
df_final.head(50)

,SKUstr,SKUint,TotalUnits,TotalRev,AWP,Category
0,32166,32166,1,1569.42,1569.420000,LENTES
1,32322,32322,1,134.89,134.890000,OTHER
2,33146,33146,1,408.00,408.000000,LENTES
3,33151,33151,1,295.00,295.000000,OBJETIVOS CÁMARAS DIGITALES
4,33582,33582,1,23.14,23.140000,OTHER
5,33583,33583,1,9.46,9.460000,OTHER
6,33588,33588,1,296.69,296.690000,LENTES
7,34924,34924,7,861.98,123.140000,OTHER
8,37679,37679,1,7.81,7.810000,CF
9,38917,38917,1,40.50,40.500000,PRISMÁTICOS


In [18]:
df_final.shape

(1270, 6)

In [19]:
df_temp = dt_feature_extraction('2017-04-01')
df_temp.columns

Index(['Date', 'date_dt', 'date_DoW', 'date_DoWnum', 'date_DoM', 'date_WoY',
       'date_isWeekend', 'date_isPublicHol', 'isSaturday'],
      dtype='object')

In [20]:
df_temp['Date']

0    2017-04-01
Name: Date, dtype: object

In [21]:
df_final.head()

,SKUstr,SKUint,TotalUnits,TotalRev,AWP,Category
0,32166,32166,1,1569.42,1569.42,LENTES
1,32322,32322,1,134.89,134.89,OTHER
2,33146,33146,1,408.00,408.00,LENTES
3,33151,33151,1,295.00,295.00,OBJETIVOS CÁMARAS DIGITALES
4,33582,33582,1,23.14,23.14,OTHER


In [23]:
for column in df_temp.columns:
    df_final[column] = df_final.apply(lambda row: df_temp[column], axis=1)

# df_final[['Date', 'date_dt', 'date_DoW', 'date_DoWnum', 
#           'date_DoM', 'date_WoY', 'date_isWeekend', 
#           'date_isPublicHol', 'isSaturday']] = df_final.apply( 
#     lambda row: pd.Series(dt_feature_extraction('2017-04-01')), axis=1)
df_final

,SKUstr,SKUint,TotalUnits,TotalRev,AWP,Category,Date,date_dt,date_DoW,date_DoWnum,date_DoM,date_WoY,date_isWeekend,date_isPublicHol,isSaturday
0,32166,32166,1,1569.42,1569.42,LENTES,2017-04-01,2017-04-01,Saturday,5,1,13,1,0,1
1,32322,32322,1,134.89,134.89,OTHER,2017-04-01,2017-04-01,Saturday,5,1,13,1,0,1
2,33146,33146,1,408.00,408.00,LENTES,2017-04-01,2017-04-01,Saturday,5,1,13,1,0,1
3,33151,33151,1,295.00,295.00,OBJETIVOS CÁMARAS DIGITALES,2017-04-01,2017-04-01,Saturday,5,1,13,1,0,1
4,33582,33582,1,23.14,23.14,OTHER,2017-04-01,2017-04-01,Saturday,5,1,13,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1265,3164880,3164880,48,1944.00,40.50,OTHER,2017-04-01,2017-04-01,Saturday,5,1,13,1,0,1
1266,3165735,3165735,2,3460.76,1730.38,OTHER,2017-04-01,2017-04-01,Saturday,5,1,13,1,0,1
1267,3169821,3169821,1,365.39,365.39,OTHER,2017-04-01,2017-04-01,Saturday,5,1,13,1,0,1
1268,S080501_500_30_EUR,3169822,123,120.54,0.98,OTHER,2017-04-01,2017-04-01,Saturday,5,1,13,1,0,1
